In [ ]:
import polars   as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()
from  rtsvg                   import PolarsSpringLayout
from  linknode_graph_patterns import LinkNodeGraphPatterns
import random
import copy
import time
def graphToDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _node_ in _g_.nodes(): 
        for _nbor_ in _g_.neighbors(_node_):
            _lu_['fm'].append(_node_), _lu_['to'].append(_nbor_)
    return pl.DataFrame(_lu_), [('fm','to')]

_patterns_ = LinkNodeGraphPatterns()
_tiles_    = []
for _type_ in _patterns_.types:
    _g_             = _patterns_.createPattern(_type_)
    _df_, _relates_ = graphToDataFrame(_g_)
    _pos1_ = nx.spring_layout(_g_)
    _pos2_ = rt.springLayout(_g_, use_weights=True)
    _pos3_ = PolarsSpringLayout(_g_).results()
    _tiles_.append(rt.link(_df_, _relates_, _pos1_))
    _tiles_.append(rt.link(_df_, _relates_, _pos2_))
    _tiles_.append(rt.link(_df_, _relates_, _pos3_))
rt.table(_tiles_, per_row=3, spacer=10)